# Bioequivalence

Bioequivalence.jl is a package for performing bioequivalence analysis part of the PumasAI ecosystem for pharmaceutical science (e.g. Pumas.jl: the simulation engine for PKPD, PBPK, QSP, and other models).

The documentation provides a how-to and technical details for performing bioequivalence analysis using various designs.

## Setup

Using JuliaPro,

### Installing

```
using Pkg;
Pkg.add("Bioequivalence")
```

### Loading

```
using Bioequivalence
```

## API

```
BioequivalenceStudy(data::AbstractDataFrame,
                    endpoint::Union{Integer, Symbol} = :AUC;
                    id::Union{Integer, Symbol} = :id,
                    sequence::Union{Integer, Symbol} = :sequence,
                    period::Union{Integer, Symbol} = :period,
                    reference::Union{Nothing, Char} = nothing,
                    nonparametric::Bool = occursin(r"(?i)tmax", string(endpoint))),
                    reml::Bool = false)
```

Return a bioequivalence study

Arguments

- data: must have id, sequence, period, and an endpoint.
- id (e.g., 1)
- sequence (e.g., RT which means first R and T in the second period)
- period (e.g, 1:4)
- endpoint: which variable is the endpoint?
- id: which variable is the subject id?
- sequence: which variable is the sequence?
- period: which variable is the period?
- reference: which formulation is the reference?

For example, in a design with RTTR|TRRT one can specify 'R' to be the reference. By default, the reference is taken to be the first character (alphabetically).

- nonparametric: use nonparametric method for analysis of the endpoint?
- reml: if the design uses a linear mixed model should it optimize REML instead of ML?

Each bioequivalence study has the following fields:

- data: data used for the study
- design: number of subjects in each sequence
- model: statistical models used for the analysis
- result: result based on previous components

Current designs include:
- Parallel (e.g., R|T, A|B|C)
- 2x2 (e.g., RT|TR)
- Balaam (e.g., RR|RT|TR|TT)
- Dual (e.g., RTT|TRR)
- 2S4P1 (e.g., RTTR|TRRT)
- 2S4P2 (e.g., RTRT|TRTR)
- WD3F (e.g., ABC|ACB|BAC|BCA|CAB|CBA)
- WD4F (e.g., ABCD|CADB|DCBA|BDAC)

## Designs

### Housekeeping

Loading a few packages to showcase the features provided by `Bioequivalence`

In [ ]:
using CSV, DataFrames, StatsBase, Bioequivalence

### Nonparametric

Nonparametric designs are included for analysis of endpoints such as Tmax.

In [ ]:
PJ31 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Nonparametric/PJ2006_3_1.tsv"))
first(PJ31, 6)

The average bioequivalence analysis can be requested through `BioequivalenceStudy`

In [ ]:
Tmax = read_be(PJ31, :Tmax)

When the name of the endpoint matches some form of Tmax (case insensitive) it defaults to nonparametric.

Otherwise, it will attempt a parametric design. This can be overwriten through the `nonparametric` argument.

Nonparametric analysis uses the Wilcoxon signed rank test.

In [ ]:
Tmax.model

Other endpoints can potentially be analyzed through a nonparametric method as well

In [ ]:
PJ46 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Williams/PJ2006_4_6.tsv"))
first(PJ46, 6)

In [ ]:
NP = read_be(PJ46, nonparametric = true)

### Parallel

Consider a parallel design dataset with balance between treatment groups such as Clayton and Leslie ([1981](https://doi.org/10.1177/030006058100900608)).

In [ ]:
ClaytonandLeslie1981 = CSV.read(string(dirname(pathof(Bioequivalence)),
                                       "/../data/Parallel/FSL2015_1.tsv"))
first(ClaytonandLeslie1981, 6)

The bioequivalence analysis can be requested through `BioequivalenceStudy` or its alias `read_be`.

In [ ]:
# notice it defaults to the AUC endpoint
Parallel = read_be(ClaytonandLeslie1981)

The output shows the design, statistical model, and result.

One can access the statistical model directly through

In [ ]:
Parallel.model

The results can be accessed through

In [ ]:
Parallel.result

or calling `coeftable` on the object

In [ ]:
coeftable(Parallel)

Notice that the results are validated with those reported in Fuglsang, Schütz, and Labes ([2015](https://doi.org/10.1208/s12248-014-9704-6)). The Geometric Means Ratio (GMR) has a point estimate of (48.58) and a 90% confidence interval of (26.78, 88.14). These are the values reported in the study obtained with various statistical packages with the Welch correction.

### Crossover

The most common bioequivalence design is perhaps the 2x2 crossover (RT|TR). Consider a dataset from Schütz, Labes, and Fuglsang ([2014](https://doi.org/10.1208/s12248-014-9661-0)) which has been simulated with an extreme range in raw data, outliers, and imbalance between sequences and a large number of subjects.

In [ ]:
SLF2014 = CSV.read(string(dirname(pathof(Bioequivalence)),
                          "/../data/2S2P/SLF2014_8.tsv"))
first(SLF2014, 6)

The bioequivalence analysis can be requested through `BioequivalenceStudy`

In [ ]:
# notice it defaults to the AUC endpoint
Crossover = read_be(SLF2014)

As with other designs one can access the specific elements of the models use in the analysis.

In [ ]:
loglikelihood(Crossover.model.model)

The results are obtained through,

In [ ]:
Crossover.result

Crossover 2x2 designs are validated with Schütz, Labes, and Fuglsang ([2014](https://doi.org/10.1208/s12248-014-9661-0)) and Patterson and Jones (2006). In this example, the estimate for the GMR is (93.42) with a 90% confidence interval of (86.81, 100.55).

### Balaam

The Balaam design (RR|RT|TR|TT) is explored with a dataset from Chow and Liu (2009).

In [ ]:
ChowandLiu2009 = CSV.read(string(dirname(pathof(Bioequivalence)),
                                 "/../data/Balaam/CL2009_9_2_1.tsv"))
first(ChowandLiu2009, 6)

The data has the same specification as other crossover studies.

The analysis follows similarly as well.

In [ ]:
# notice it defaults to the AUC endpoint
Balaam = read_be(ChowandLiu2009)

We can also obtain the results for the intra-subject variabilities through

In [ ]:
Balaam.model.σ

Designs with repeated measures include rescaled parameter estimates and variance analysis.

The rescaled parameter estimates are especially of interest when the reference drug is a highly variable drug (HVD) or a narrow therapeutic index (NTI) drug (Haidar et al. 2008).

The default values are:

- regulatory constant for reference-scaled estimates: 0.1
- auxiliary parameter for reference-scaled estimates: 1.11

These maybe be overwriten by passing the keyword arguments (see the docstring of `read_be` for more details).

### Dual

Consider the example 4.1 in Patterson and Jones (2006)

In [ ]:
PJ41 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Dual/PJ2006_4_1.tsv"))
first(PJ41, 6)

In [ ]:
Dual = read_be(PJ41, :Cmax)

### 2S4P Designs

For 2S4P designs both RRTT|TTRR and RTRT|TRTR are supported

In [ ]:
PJ43 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/2S4P/PJ2006_4_3.tsv"))
first(PJ43, 6)

In [ ]:
Inner = read_be(PJ43)

In [ ]:
PJ44 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/2S4P/PJ2006_4_4.tsv"))
first(PJ44, 6)

In [ ]:
Outer = read_be(PJ44, :Cmax)

### Williams

Consider a 3 formulations Williams design

In [ ]:
PJ45 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Williams/PJ2006_4_5.tsv"))
first(PJ45, 6)

In [ ]:
W3F = read_be(PJ45)

Imagine for a moment that the reference formulation is actually S instead of R.
One can pass such a parameter as following.

In [ ]:
W3F = read_be(PJ45, reference = 'S')

Williams designs for four formulations are available as well

In [ ]:
PJ46 = CSV.read(string(dirname(pathof(Bioequivalence)),
                       "/../data/Williams/PJ2006_4_6.tsv"))
first(PJ46, 6)

In [ ]:
W4F = read_be(PJ46)

## References

Chow, Shein-Chung, and Jen-pei Liu. 2009. Design and Analysis of Bioavailability and Bioequivalence Studies. 3rd ed. Chapman & Hall/CRC Biostatistics Series 27. Boca Raton: CRC Press.

Clayton, D, and A Leslie. 1981. "The Bioavailability of Erythromycin Stearate versus Enteric-Coated Erythromycin Base When Taken Immediately before and after Food." Journal of International Medical Research 9 (6): 470-77. [DOI:10.1177/030006058100900608](https://doi.org/10.1177/030006058100900608).

Fuglsang, Anders, Helmut Schütz, and Detlew Labes. 2015. "Reference Datasets for Bioequivalence Trials in a Two-Group Parallel Design." The AAPS Journal 17 (2): 400-404. [DOI:10.1208/s12248-014-9704-6](https://doi.org/10.1208/s12248-014-9704-6).

Haidar, Sam H., Fairouz Makhlouf, Donald J. Schuirmann, Terry Hyslop, Barbara Davit, Dale Conner, and Lawrence X. Yu. 2008. "Evaluation of a Scaling Approach for the Bioequivalence of Highly Variable Drugs." The AAPS Journal 10 (3): 450-54. [DOI:10.1208/s12248-008-9053-4](https://doi.org/10.1208/s12248-008-9053-4).

Patterson, Scott D, and Byron Jones. 2006. Bioequivalence and Statistics in Clinical Pharmacology. Boca Raton: Chapman & Hall/CRC. ISBN: 9781420034936.

Schütz, Helmut, Detlew Labes, and Anders Fuglsang. 2014. "Reference Datasets for 2-Treatment, 2-Sequence, 2-Period Bioequivalence Studies." The AAPS Journal 16 (6): 1292-97. [DOI:10.1208/s12248-014-9661-0](https://doi.org/10.1208/s12248-014-9661-0).